# Generic NN trainer

In [1]:
import sys
from os.path import splitext, basename

import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.simplefilter("ignore")

%load_ext lab_black

sys.path.append("/Users/renero/Documents/SideProjects/trader/src")
sys.path.append("/Users/renero/Documents/SideProjects/trader/src/predictor")


from dictionary import Dictionary
from lstm import lstm
from predictor.ticks import Ticks
from utils.utils import print_bin_predictions_match
from utils.plot_utils import plot_marks

## Setup the generic parameters for the trainer

In [2]:
argv = [
    "",
    "-c",
    "../params/params.eurusd.yaml",
    "-f",
    "../data/EURUSD/EURUSD_2018_2020.csv",
    "--window",
    "14",
    "--epochs",
    "100",
    "train",
]
params = Dictionary(args=argv)


def read_ticks():
    global params

    ticks = Ticks(params, csv_file=params.input_file).scale()
    ticks.append_indicator(["trend", "median_filter", "change"])
    ticks.append_indicator("gmf", monotonic_window=7, mf_window=3, sigma=5)
    ticks.append_indicator("gmf_mono", monotonic_window=7, mf_window=3, sigma=5)
    ticks.append_indicator("trend", column_name="gmf")
    return ticks


ticks = read_ticks()
ticks.data.head()

2020-10-25 17:30:21 - INFO  - Dictionary:__init__             - Using configuration parameters from: ../params/params.eurusd.yaml


,open,high,low,close,close_trend,med_filter,change,returns,gmf,gmf_mono,gmf_trend
Datetime,,,,,,,,,,,
2018-01-01,1.2274,1.1598,1.2794,1.2442,1.0,1.2423,0.0000,0.000000,0.001247,0.0,1.0
2018-01-02,1.2292,1.2731,1.2922,1.3358,1.0,1.2442,0.0916,0.073622,0.001905,0.0,1.0
2018-01-03,1.3218,1.2430,1.2794,1.2423,0.0,1.2753,-0.0935,-0.069996,0.003268,0.0,1.0
2018-01-04,1.2292,1.2873,1.2849,1.3449,1.0,1.2918,0.1026,0.082589,0.005403,0.0,1.0
2018-01-05,1.3309,1.2767,1.3160,1.2753,0.0,1.2753,-0.0696,-0.051751,0.008347,0.0,1.0


In [3]:
# plot_marks(data=ticks.data, signal="close", marks="gmf_mono", dark=True)
# plot_marks(data=ticks.data, signal="close", marks="gmf_trend", dark=True)
# plot_marks(data=ticks.data, signal="gmf", marks="gmf_mono", dark=True)

---

# Predict CLOSE with GMF, MONOTONIC SIGN and GMF TREND

```
       _                           _                      _ 
      | |                         | |                    | |
   ___| | ___  ___  ___           | |_ _ __ ___ _ __   __| |
  / __| |/ _ \/ __|/ _ \          | __| '__/ _ \ '_ \ / _` |
 | (__| | (_) \__ \  __/          | |_| | |  __/ | | | (_| |
  \___|_|\___/|___/\___|           \__|_|  \___|_| |_|\__,_|
                          ______                            
                         |______|                           
```

.

In [ ]:
ticks = read_ticks()
X_train_close, y_train_close, X_test_close, y_test_close = ticks.prepare_for_training(
    predict_column="close_trend", train_columns=["gmf", "gmf_mono", "gmf_trend"]
)

nn1 = lstm(params).build()
nn1.start_training(X_train_close, y_train_close, name=None)
yhat_close_trend, acc = nn1.evaluate(X_test_close, y_test_close)

In [ ]:
print(nn1)
print_bin_predictions_match(y_test_close, yhat_close_trend)

# Save the model and the scaler!

In [ ]:
params.models_dir = "../staging/EURUSD/"
nn1.save()

## Save the SCALER

In [ ]:
# dataset_name = splitext(basename(params.input_file))[0]
# ticks.save_scaler()

## Make prediction with the loaded model

In [ ]:
yhat_close_trend, acc_close_trend = nn1.evaluate(X_train_close, y_train_close)

----

# Predict GMF TREND with GMF and MONOTONIC SIGN

```
                   __            _                      _ 
                  / _|          | |                    | |
   __ _ _ __ ___ | |_           | |_ _ __ ___ _ __   __| |
  / _` | '_ ` _ \|  _|          | __| '__/ _ \ '_ \ / _` |
 | (_| | | | | | | |            | |_| | |  __/ | | | (_| |
  \__, |_| |_| |_|_|             \__|_|  \___|_| |_|\__,_|
   __/ |                ______                            
  |___/                |______|                           
```

---

In [11]:
ticks = read_ticks()
X_train_gmf, y_train_gmf, X_test_gmf, y_test_gmf = ticks.prepare_for_training(
    predict_column="gmf_trend", train_columns=["gmf", "gmf_mono"]
)

nn2 = lstm(params).build()
nn2.start_training(X_train_gmf, y_train_gmf, name=None)
yhat, acc = nn2.evaluate(X_test_gmf, y_test_gmf)

2020-10-25 17:37:53 - INFO  - lstm:build                      - NN lstm_1layers created
2020-10-25 17:37:53 - INFO  - lstm:start_training             - Training for 150 epochs...
Epoch 150/150 - Acc:0.92 (↑0.94/↓0.73) - Val:0.73 (↑0.84/↓0.60) | [====================]
3/3 [==============================] - 0s 2ms/step - loss: 0.5826 - accuracy: 0.6988
2020-10-25 17:39:06 - INFO  - lstm:evaluate                   - Predictions (yhat): (83, 1)
2020-10-25 17:39:06 - INFO  - lstm:evaluate                   - Accuracy: 0.70
2020-10-25 17:39:06 - INFO  - lstm:evaluate                   - Trend acc.: 0.70


## Save the model

In [12]:
print(nn2)
print_bin_predictions_match(y_test_gmf, yhat)

nn2.save()

LSTMb_gmf_trend;1L;u28;d0.20;lr0.01;W28;E150;BS8

00 | 0 | 0.24 | 1 | 0.24 | 1 | 0.78 | 1 | 0.97 | 1 | 0.99 | 1 | 0.99 | 1 | 0.99 | 1 | 0.98 | 1 | 0.96 | 
09 | 1 | 0.90 | 1 | 0.07 | 1 | 0.14 | 1 | 0.08 | 1 | 0.18 | 1 | 0.08 | 1 | 0.10 | 1 | 0.15 | 1 | 0.07 | 
18 | 1 | 0.10 | 1 | 0.26 | 1 | 0.09 | 1 | 0.31 | 1 | 0.24 | 1 | 0.34 | 1 | 0.30 | 1 | 0.25 | 1 | 0.22 | 
27 | 1 | 0.19 | 1 | 0.19 | 1 | 0.40 | 1 | 0.29 | 1 | 0.18 | 1 | 0.12 | 1 | 0.10 | 0 | 0.09 | 0 | 0.03 | 
36 | 0 | 0.01 | 0 | 0.01 | 0 | 0.00 | 0 | 0.00 | 0 | 0.00 | 0 | 0.00 | 0 | 0.00 | 0 | 0.00 | 0 | 0.00 | 
45 | 0 | 0.00 | 0 | 0.00 | 0 | 0.00 | 0 | 0.00 | 0 | 0.01 | 0 | 0.01 | 0 | 0.01 | 0 | 0.01 | 0 | 0.01 | 
54 | 0 | 0.01 | 0 | 0.01 | 0 | 0.01 | 0 | 0.01 | 0 | 0.02 | 0 | 0.02 | 0 | 0.02 | 0 | 0.02 | 0 | 0.03 | 
63 | 0 | 0.03 | 0 | 0.03 | 0 | 0.04 | 0 | 0.04 | 0 | 0.04 | 0 | 0.05 | 0 | 0.05 | 0 | 0.05 | 0 | 0.05 | 
72 | 0 | 0.06 | 0 | 0.06 | 0 | 0.07 | 0 | 0.08 | 0 | 0.10 | 0 | 0.14 | 0 | 0.21 | 0 | 0.28 | 0 | 0.25 | 
81 | 

----

# Predict GMF MONO with GMF and GMF_TREND

```
                   __                                        
                  / _|                                       
   __ _ _ __ ___ | |_            _ __ ___   ___  _ __   ___  
  / _` | '_ ` _ \|  _|          | '_ ` _ \ / _ \| '_ \ / _ \ 
 | (_| | | | | | | |            | | | | | | (_) | | | | (_) |
  \__, |_| |_| |_|_|            |_| |_| |_|\___/|_| |_|\___/ 
   __/ |                ______                               
  |___/                |______|                             
```

..

In [ ]:
ticks = read_ticks()
params.epochs = 100
params.window_size = 42

X_train, y_train, X_test, y_test = ticks.prepare_for_training(
    predict_column="gmf_mono", train_columns=["gmf", "gmf_trend"]
)

nn3 = lstm(params).build()
print(nn3)
nn3.start_training(X_train, y_train, name=None)
yhat, acc = nn3.evaluate(X_test, y_test)

## Save the model

In [ ]:
print_bin_predictions_match(y_test, yhat)
# nn3.save()

## Evaluate on training

In [ ]:
yhat3, acc3 = nn3.evaluate(X_train , y_train)

---

# Ensemble of the output from the two networkss

```
                                _     _         _______      ____  __ 
                               | |   | |       / ____\ \    / /  \/  |
   ___ _ __  ___  ___ _ __ ___ | |__ | | ___  | (___  \ \  / /| \  / |
  / _ \ '_ \/ __|/ _ \ '_ ` _ \| '_ \| |/ _ \  \___ \  \ \/ / | |\/| |
 |  __/ | | \__ \  __/ | | | | | |_) | |  __/  ____) |  \  /  | |  | |
  \___|_| |_|___/\___|_| |_| |_|_.__/|_|\___| |_____/    \/   |_|  |_|
                                                                      
                                                                      
```
.

In [ ]:
import pandas as pd

ensemble = pd.DataFrame(
    {
        "yhat_close": yhat_close_trend.reshape(-1,),
        "yhat_gmf": yhat_gmf_trend.reshape(-1,),
        "y_close": y_train_close.reshape(-1,),
        "y_gmf": y_train_gmf.reshape(-1,),
    }
)
ensemble.head()

Entrenar la SVM con los datos usados para el entrenamiento de las LSTM, pero hacer la validación con los datos de test.

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

svclassifier = SVC(kernel="rbf")
svclassifier.fit(X, y)

yhat_close_test, _ = nn1.evaluate(X_test_close, y_test_gmf)
yhat_gmf_test, _ = nn2.evaluate(X_test_gmf, y_test_gmf)

X_test = pd.DataFrame(
    {
        "yhat_close": yhat_close_test.reshape(-1,),
        "yhat_gmf": yhat_gmf_test.reshape(-1,),
    }
)
y_test = y_test_close.reshape(-1,)

In [ ]:
y_pred = svclassifier.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))